In [10]:
import os
import tkinter as tk
from tkinter import filedialog

import numpy as np
import ezc3d
import sqlite3
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

# ===================== Discontinuity Resolution ===================== #
def resolve_angle_discontinuities(angle_array, threshold=180.0):
    """
    Resolves angle discontinuities in degrees.
    If the difference between consecutive frames is > 'threshold',
    shifts subsequent angles by ±360° to keep them continuous.
    """
    angles = angle_array.copy()
    for i in range(1, len(angles)):
        diff = angles[i] - angles[i-1]
        if diff > threshold:
            angles[i:] -= 360.0
        elif diff < -threshold:
            angles[i:] += 360.0
    return angles

# ===================== SQL Setup ===================== #
DB_PATH = "pitch_analysis.sqlite"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS pitch_kinematics (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  file_name TEXT,
  participant_name TEXT,
  pitch_date TEXT,
  trial_type TEXT,
  foot_contact_frame INTEGER,
  ball_release_frame INTEGER,
  peak_torso_vel REAL,
  peak_humerus_vel REAL,
  peak_hand_vel REAL,
  max_abduction REAL,
  abd_foot_contact REAL,
  er_foot_contact REAL,
  max_er REAL
)
""")
conn.commit()
conn.close()

# ===================== Tkinter Folder Selection ===================== #
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Folder Containing C3D Files")
if not selected_folder:
    raise ValueError("No folder was selected.")

c3d_files = [os.path.join(selected_folder, f) for f in os.listdir(selected_folder)
             if f.lower().endswith(".c3d")]

if not c3d_files:
    raise ValueError("No .c3d files found in the selected folder!")

static_files = [f for f in c3d_files if "static" in os.path.basename(f).lower()]
if not static_files:
    print("No static trial found. Using raw angles without normalization.")
else:
    print("Found static files:")
    for sf in static_files:
        print("  ", os.path.basename(sf))

# ===================== Functions to Load & Process C3D ===================== #
def load_c3d_file(c3d_path):
    """
    Loads a C3D file and extracts marker data.
    Returns: c3d object, markers dict {marker_name: Nx3 array}, frame_rate, n_frames, hand_side.
    """
    c3d_obj = ezc3d.c3d(c3d_path)
    points = c3d_obj["data"]["points"]
    labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    n_frames = points.shape[2]

    print(f"\nAvailable Markers in {os.path.basename(c3d_path)}:")
    print(labels)

    # Remove extra spaces and convert to upper case
    markers = {lab.strip().upper(): points[:3, i, :].T for i, lab in enumerate(labels)}

    hand_side = "Right" if any("RIGHT_" in lab.strip().upper() for lab in labels) else "Left"
    return c3d_obj, markers, frame_rate, n_frames, hand_side

def detect_foot_contact_and_ball_release(events, markers, frame_rate):
    """
    Returns foot_contact_frame and ball_release_frame.
    (Uses events if available or falls back to a heuristic using foot markers.)
    """
    fc_frame = None
    br_frame = None
    for lab, val in events.items():
        if "foot" in lab and "contact" in lab:
            fc_frame = val
        if "ball" in lab and "release" in lab:
            br_frame = val
    if fc_frame is None:
        possible_foot_markers = [m for m in markers if "HEEL" in m or "TOE" in m or "FOOT" in m]
        if possible_foot_markers:
            fm = possible_foot_markers[0]
            Z = markers[fm][:, 2]
            vz = np.gradient(Z) * frame_rate
            for i in range(1, len(Z)):
                if vz[i-1] < 0 and vz[i] >= 0:
                    fc_frame = i
                    break
    if fc_frame is None:
        fc_frame = 0
    if br_frame is None:
        ball_marker = [m for m in markers if "BALL" in m]
        hand_marker = [m for m in markers if "WRIST" in m or "HAND" in m]
        if ball_marker and hand_marker:
            b = ball_marker[0]
            h = hand_marker[0]
            dist = np.linalg.norm(markers[b] - markers[h], axis=1)
            for i in range(1, len(dist)):
                if dist[i] > 0.03:
                    br_frame = i
                    break
    if br_frame is None or br_frame < fc_frame:
        br_frame = fc_frame + int(frame_rate * 0.5)
    return fc_frame, br_frame

def compute_torso_humerus_hand_kinematics(markers, frame_rate, static_offsets=None):
    print("\nProcessing markers for joint angle calculations:")
    available_keys = list(markers.keys())
    print(available_keys)
    
    # Check for either naming convention for the right shoulder
    if "RIGHT_SHOULDER" in available_keys or "R_SHOULDER" in available_keys:
        hand_side = "RIGHT"
    else:
        hand_side = "LEFT"

    if hand_side == "RIGHT":
        required_keys = ["L_SHOULDER", "R_SHOULDER", "RIGHT_LATERAL_ELBOW", "RIGHT_MEDIAL_ELBOW", "RIGHT_WRIST_RADIUS", "RIGHT_WRIST_ULNA", "RIGHT_HAND"]
    else:
        required_keys = ["L_SHOULDER", "R_SHOULDER", "LEFT_LATERAL_ELBOW", "LEFT_MEDIAL_ELBOW", "LEFT_WRIST_RADIUS", "LEFT_WRIST_ULNA", "LEFT_HAND"]

    # Debug: Check which required key is missing
    for key in required_keys:
        if key not in markers:
            print(f"Required marker {key} is missing in this trial.")
    
    if not all(key in markers for key in required_keys):
        print("Missing required markers! Skipping this trial.")
        return None

    # Retrieve markers based on hand_side
    RSHO = markers.get("R_SHOULDER") if hand_side == "RIGHT" else None
    LSHO = markers.get("L_SHOULDER")
    if hand_side == "RIGHT":
        ELB_LAT = markers.get("RIGHT_LATERAL_ELBOW")
        ELB_MED = markers.get("RIGHT_MEDIAL_ELBOW")
        WRI_RAD = markers.get("RIGHT_WRIST_RADIUS")
        WRI_ULNA = markers.get("RIGHT_WRIST_ULNA")
        HAND    = markers.get("RIGHT_HAND")
    else:
        ELB_LAT = markers.get("LEFT_LATERAL_ELBOW")
        ELB_MED = markers.get("LEFT_MEDIAL_ELBOW")
        WRI_RAD = markers.get("LEFT_WRIST_RADIUS")
        WRI_ULNA = markers.get("LEFT_WRIST_ULNA")
        HAND    = markers.get("LEFT_HAND")


    # Compute centers and vectors
    ELB = (ELB_LAT + ELB_MED) / 2.0
    WRI = (WRI_RAD + WRI_ULNA) / 2.0
    humerus_vec = ELB - RSHO         # from shoulder to elbow
    forearm_vec = WRI - ELB          # from elbow to wrist
    torso_vec   = RSHO - LSHO        # shoulder-to-shoulder vector

    # Shoulder abduction: angle between humerus vector and vertical [0,0,1]
    hum_norm = np.linalg.norm(humerus_vec, axis=1, keepdims=True) + 1e-8
    hum_unit = humerus_vec / hum_norm
    vertical = np.array([0, 0, 1])
    dot_vals = np.sum(hum_unit * vertical, axis=1)
    dot_vals = np.clip(dot_vals, -1, 1)
    shoulder_abduction = np.degrees(np.arccos(dot_vals))

    # Shoulder external rotation:
    dot_fore = np.sum(forearm_vec * hum_unit, axis=1, keepdims=True)
    fore_proj = forearm_vec - dot_fore * hum_unit
    fore_norm = np.linalg.norm(fore_proj, axis=1, keepdims=True) + 1e-8
    fore_unit = fore_proj / fore_norm
    ref = np.array([1, 0, 0])
    dot_ref = np.sum(ref * hum_unit, axis=1, keepdims=True)
    ref_proj = ref - dot_ref * hum_unit
    ref_norm = np.linalg.norm(ref_proj, axis=1, keepdims=True) + 1e-8
    ref_unit = ref_proj / ref_norm
    dot_ex = np.sum(fore_unit * ref_unit, axis=1)
    dot_ex = np.clip(dot_ex, -1, 1)
    shoulder_external_rot = np.degrees(np.arccos(dot_ex))

    shoulder_abduction = resolve_angle_discontinuities(shoulder_abduction)
    shoulder_external_rot = resolve_angle_discontinuities(shoulder_external_rot)

    torso_ang_vel = np.gradient(shoulder_abduction) * frame_rate
    humerus_ang_vel = np.gradient(shoulder_abduction) * frame_rate
    hand_ang_vel = np.gradient(shoulder_external_rot) * frame_rate

    return {
        "shoulder_abduction": shoulder_abduction,
        "shoulder_external_rot": shoulder_external_rot,
        "torso_vel": torso_ang_vel,
        "humerus_vel": humerus_ang_vel,
        "hand_vel": hand_ang_vel
    }

def compute_static_offsets(static_file):
    """ Computes static offsets from the static trial to normalize dynamic angles. """
    _, markers, frame_rate, _, _ = load_c3d_file(static_file)
    kin = compute_torso_humerus_hand_kinematics(markers, frame_rate)
    if kin is None:
        print(f"WARNING: Could not compute static offsets from {static_file}")
        return {}
    mean_abd = np.mean(kin["shoulder_abduction"])
    mean_er  = np.mean(kin["shoulder_external_rot"])
    print(f"Static Offsets -> Abduction: {mean_abd:.2f}, External Rotation: {mean_er:.2f}")
    return {"abd": mean_abd, "er": mean_er}

# ===================== Compute Static Offsets if available ===================== #
static_offsets = {}
if static_files:
    static_offsets = compute_static_offsets(static_files[0])
    print("Static Offsets:", static_offsets)

# ===================== MAIN PROCESSING LOOP ===================== #
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

for file_path in c3d_files:
    base_name = os.path.basename(file_path)
    if "static" in base_name.lower():
        continue

    print(f"\nProcessing {base_name}")
    try:
        c3d_obj, markers, frame_rate, n_frames, hand_side = load_c3d_file(file_path)
    except Exception as e:
        print(f"Error loading {base_name}: {e}")
        continue

    fc_frame = 0
    br_frame = n_frames - 1

    kin = compute_torso_humerus_hand_kinematics(markers, frame_rate, static_offsets)
    if kin is None:
        print(f"Skipping {base_name}; could not compute angles.")
        continue

    peak_torso = np.max(kin["torso_vel"])
    peak_hum   = np.max(kin["humerus_vel"])
    peak_hand  = np.max(kin["hand_vel"])
    abd_in_window = kin["shoulder_abduction"][fc_frame:br_frame+1]
    max_abd = np.max(abd_in_window)
    abd_fc = kin["shoulder_abduction"][fc_frame]
    er_fc = kin["shoulder_external_rot"][fc_frame]
    max_er = np.max(kin["shoulder_external_rot"][fc_frame:br_frame+1])

    folder_parts = os.path.normpath(file_path).split(os.sep)
    participant_name = folder_parts[-3] if len(folder_parts) >= 3 else "Unknown"
    pitch_date = folder_parts[-2].replace("_", "") if len(folder_parts) >= 3 else "Unknown"
    trial_type = os.path.splitext(folder_parts[-1])[0] if len(folder_parts) >= 1 else "Unknown"

    insert_sql = """
    INSERT INTO pitch_kinematics (
        file_name, participant_name, pitch_date, trial_type,
        foot_contact_frame, ball_release_frame,
        peak_torso_vel, peak_humerus_vel, peak_hand_vel,
        max_abduction, abd_foot_contact, er_foot_contact, max_er
    ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)
    """
    data_tuple = (
        base_name,
        participant_name,
        pitch_date,
        trial_type,
        fc_frame,
        br_frame,
        float(peak_torso),
        float(peak_hum),
        float(peak_hand),
        float(max_abd),
        float(abd_fc),
        float(er_fc),
        float(max_er)
    )
    cursor.execute(insert_sql, data_tuple)
    conn.commit()

conn.close()
print("\nDone processing all trials. Data stored in pitch_kinematics table.")

# ===================== BUILD DASH APP ===================== #
external_stylesheets = [dbc.themes.BOOTSTRAP, "assets/custom.css"]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Youth Pitch Design Kinematics"

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Pitch Kinematics", className="text-center text-primary mb-4"), width=12)
    ]),
    dbc.Row([
        dbc.Col([
            html.Label("Participant"),
            dcc.Dropdown(id="participant-dropdown", style={"margin-bottom": "20px"}),
            html.Label("Date"),
            dcc.Dropdown(id="date-dropdown", style={"margin-bottom": "20px"}),
            html.Label("Trial Type"),
            dcc.Dropdown(id="trial-dropdown", style={"margin-bottom": "20px"}),
            dbc.Button("Refresh Data", id="refresh-btn", color="primary", className="w-100")
        ], width=3),
        dbc.Col([
            dcc.Graph(id="velocity-graph")
        ], width=9)
    ])
], fluid=True)

@app.callback(
    Output("participant-dropdown", "options"),
    Output("participant-dropdown", "value"),
    Input("refresh-btn", "n_clicks")
)
def update_participants(_):
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_kinematics", conn)
    conn.close()
    participants = df["participant_name"].unique().tolist()
    options = [{"label": p, "value": p} for p in participants]
    val = participants[0] if participants else None
    return options, val

@app.callback(
    Output("date-dropdown", "options"),
    Output("date-dropdown", "value"),
    Input("participant-dropdown", "value")
)
def update_dates(participant):
    if not participant:
        return [], None
    conn = sqlite3.connect(DB_PATH)
    query = f"SELECT DISTINCT pitch_date FROM pitch_kinematics WHERE participant_name='{participant}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    dates = df["pitch_date"].unique().tolist()
    options = [{"label": d, "value": d} for d in dates]
    return options, dates[0] if dates else None

@app.callback(
    Output("trial-dropdown", "options"),
    Output("trial-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value")
)
def update_trial_types(participant, date):
    if not participant or not date:
        return [], None
    conn = sqlite3.connect(DB_PATH)
    query = f"SELECT DISTINCT trial_type FROM pitch_kinematics WHERE participant_name='{participant}' AND pitch_date='{date}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    trials = df["trial_type"].unique().tolist()
    options = [{"label": t, "value": t} for t in trials]
    return options, trials[0] if trials else None

@app.callback(
    Output("velocity-graph", "figure"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value"),
    Input("trial-dropdown", "value")
)
def update_graph(participant, date, trial):
    if not participant or not date or not trial:
        return go.Figure()
    conn = sqlite3.connect(DB_PATH)
    query = f"""
    SELECT file_name, peak_torso_vel, peak_humerus_vel, peak_hand_vel
    FROM pitch_kinematics
    WHERE participant_name='{participant}' AND pitch_date='{date}' AND trial_type='{trial}'
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df["file_name"], y=df["peak_torso_vel"], name="Peak Torso Vel"))
    fig.add_trace(go.Bar(x=df["file_name"], y=df["peak_humerus_vel"], name="Peak Humerus Vel"))
    fig.add_trace(go.Bar(x=df["file_name"], y=df["peak_hand_vel"], name="Peak Hand Vel"))
    fig.update_layout(barmode='group',
                      title="Peak Angular Velocities",
                      xaxis_title="Trial",
                      yaxis_title="Velocity (deg/s)")
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)


Found static files:
   A+ Static RH 2.c3d

Available Markers in A+ Static RH 2.c3d:
['L_Shoulder', 'Sternum', 'R_Shoulder', 'C7', 'Bicep', 'Tricep', 'Right_Lateral_Elbow', 'Right_Medial_Elbow', 'Forearm', 'Right_Wrist_Radius', 'Right_Wrist_Ulna', 'Right_Hand', 'Waist']

Processing markers for joint angle calculations:
['L_SHOULDER', 'STERNUM', 'R_SHOULDER', 'C7', 'BICEP', 'TRICEP', 'RIGHT_LATERAL_ELBOW', 'RIGHT_MEDIAL_ELBOW', 'FOREARM', 'RIGHT_WRIST_RADIUS', 'RIGHT_WRIST_ULNA', 'RIGHT_HAND', 'WAIST']
Static Offsets -> Abduction: 114.76, External Rotation: 99.41
Static Offsets: {'abd': np.float64(114.7626317982866), 'er': np.float64(99.41296666463018)}

Processing Crow Hop RH 4.c3d

Available Markers in Crow Hop RH 4.c3d:
['L_Shoulder', 'Sternum', 'R_Shoulder', 'C7', 'Bicep', 'Tricep', 'Right_Lateral_Elbow', 'Right_Medial_Elbow', 'Forearm', 'Right_Wrist_Radius', 'Right_Wrist_Ulna', 'Right_Hand', 'Waist']

Processing markers for joint angle calculations:
['L_SHOULDER', 'STERNUM', 'R_SHOU

In [4]:
import os
import tkinter as tk
from tkinter import filedialog

import numpy as np
import ezc3d
import sqlite3
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

# ===================== Discontinuity Resolution ===================== #
def resolve_angle_discontinuities(angle_array, threshold=180.0):
    """
    Resolves angle discontinuities in degrees.
    If the difference between consecutive frames is > 'threshold',
    shift subsequent angles by ±360 degrees to keep them continuous.
    """
    angles = angle_array.copy()
    for i in range(1, len(angles)):
        diff = angles[i] - angles[i-1]
        if diff > threshold:
            angles[i:] -= 360.0
        elif diff < -threshold:
            angles[i:] += 360.0
    return angles

# ===================== SQL Setup ===================== #
DB_PATH = "pitch_analysis.sqlite"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Table for saving kinematic results
cursor.execute("""
CREATE TABLE IF NOT EXISTS pitch_kinematics (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  file_name TEXT,
  participant_name TEXT,
  pitch_date TEXT,
  trial_type TEXT,
  foot_contact_frame INTEGER,
  ball_release_frame INTEGER,
  peak_torso_vel REAL,
  peak_humerus_vel REAL,
  peak_hand_vel REAL,
  max_abduction REAL,
  abd_foot_contact REAL,
  er_foot_contact REAL,
  max_er REAL
)
""")
conn.commit()
conn.close()

# ===================== Tkinter Folder Selection ===================== #
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Folder Containing C3D Files")
if not selected_folder:
    raise ValueError("No folder was selected.")

# Gather all .c3d in folder
c3d_files = [os.path.join(selected_folder, f) for f in os.listdir(selected_folder)
             if f.lower().endswith(".c3d")]

if not c3d_files:
    raise ValueError("No .c3d files found in the selected folder!")

# Identify static trial(s)
static_files = [f for f in c3d_files if "static" in os.path.basename(f).lower()]
if not static_files:
    print("No static trial found. Using raw angles without normalization.")
else:
    print("Found static files:")
    for sf in static_files:
        print("  ", os.path.basename(sf))

# ===================== Functions to Load & Process C3D ===================== #
def load_c3d_file(c3d_path):
    """ Load C3D file and extract marker data. """
    c3d_obj = ezc3d.c3d(c3d_path)
    points = c3d_obj["data"]["points"]  # shape (4, #markers, #frames)
    labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    n_frames = points.shape[2]

    # Debug: Print marker names to confirm correct mapping
    print(f"\nAvailable Markers in {os.path.basename(c3d_path)}:")
    print(labels)

    # Create dictionary with marker names
    markers = {lab.upper(): points[:3, i, :].T for i, lab in enumerate(labels)}

    # Detect hand (Left or Right)
    hand = "Right" if any("RIGHT_" in lab.upper() for lab in labels) else "Left"

    return c3d_obj, markers, frame_rate, n_frames, hand


def detect_foot_contact_and_ball_release(events, markers, frame_rate):
    """
    Return foot_contact_frame, ball_release_frame
    If events dictionary has them, use them. Otherwise, do a heuristic with markers.
    """
    fc_frame = None
    br_frame = None
    # If events exist, attempt to find foot contact and ball release by name
    for lab, val in events.items():
        if "foot" in lab and "contact" in lab:
            fc_frame = val
        if "ball" in lab and "release" in lab:
            br_frame = val
    # If not found, do a fallback method:
    # e.g., foot contact: a foot marker's vertical velocity crosses 0
    if fc_frame is None:
        possible_foot_markers = [m for m in markers if "HEEL" in m or "TOE" in m or "FOOT" in m]
        if possible_foot_markers:
            fm = possible_foot_markers[0]
            Z = markers[fm][:, 2]
            vz = np.gradient(Z) * frame_rate
            for i in range(1, len(Z)):
                if vz[i-1] < 0 and vz[i] >= 0:
                    fc_frame = i
                    break
    if fc_frame is None:
        fc_frame = 0  # default to start

    # ball release: if a 'ball' marker is present, find separation from hand
    if br_frame is None:
        ball_marker = [m for m in markers if "BALL" in m]
        hand_marker = [m for m in markers if "WRIST" in m or "HAND" in m]
        if ball_marker and hand_marker:
            b = ball_marker[0]
            h = hand_marker[0]
            dist = np.linalg.norm(markers[b] - markers[h], axis=1)
            for i in range(1, len(dist)):
                if dist[i] > 0.03:  # 3 cm threshold
                    br_frame = i
                    break
    if br_frame is None or br_frame < fc_frame:
        br_frame = fc_frame + int(frame_rate * 0.5)  # default 0.5s after FC

    return fc_frame, br_frame


def compute_torso_humerus_hand_kinematics(markers, frame_rate, static_offsets=None):
    """
    Computes joint angles and segment velocities using the correct marker names.
    """

    print("\nProcessing markers for joint angle calculations:")
    print(list(markers.keys()))  # Debugging: Print available markers

    # Correct marker names based on provided list
    RSHO = markers.get("R_Shoulder")
    LSHO = markers.get("L_Shoulder")
    RELB_Lat = markers.get("Right_Lateral_Elbow")
    RELB_Med = markers.get("Right_Medial_Elbow")
    RWR_Rad = markers.get("Right_Wrist_Radius")
    RWR_Ulna = markers.get("Right_Wrist_Ulna")
    HAND = markers.get("Right_Hand")

    if not all([RSHO, LSHO, RELB_Lat, RELB_Med, RWR_Rad, RWR_Ulna, HAND]):
        print("Missing required markers! Skipping this trial.")
        return None

    # Compute elbow center (midpoint of lateral and medial elbow)
    RELB = (RELB_Lat + RELB_Med) / 2.0

    # Compute wrist center (midpoint of radius and ulna)
    RWR = (RWR_Rad + RWR_Ulna) / 2.0

    # Compute segment vectors
    torso_vec = RSHO - LSHO  # Shoulder-to-shoulder axis
    humerus_vec = RELB - RSHO  # Shoulder to elbow
    forearm_vec = RWR - RELB  # Elbow to wrist

    # Shoulder Abduction (Angle between humerus and vertical plane)
    shoulder_abduction = np.degrees(np.arccos(np.clip(
        np.dot(humerus_vec / np.linalg.norm(humerus_vec, axis=1, keepdims=True),
               np.array([0, 0, 1])),
        -1.0, 1.0)))

    # External Rotation (Angle between forearm and humerus in the transverse plane)
    plane_normal = np.cross(humerus_vec, np.array([0, 0, 1]))  # Plane perpendicular to humerus
    hand_proj = forearm_vec - np.dot(forearm_vec, plane_normal)[:, np.newaxis] * plane_normal

    shoulder_external_rot = np.degrees(np.arccos(np.clip(
        np.dot(humerus_vec / np.linalg.norm(humerus_vec, axis=1, keepdims=True),
               hand_proj / np.linalg.norm(hand_proj, axis=1, keepdims=True)),
        -1.0, 1.0)))

    # Compute Angular Velocities
    torso_ang_vel = np.gradient(np.degrees(np.arccos(np.clip(
        np.dot(torso_vec[:-1], torso_vec[1:].T), -1.0, 1.0)))) * frame_rate
    humerus_ang_vel = np.gradient(shoulder_abduction) * frame_rate
    hand_ang_vel = np.gradient(shoulder_external_rot) * frame_rate

    return {
        "shoulder_abduction": shoulder_abduction,
        "shoulder_external_rot": shoulder_external_rot,
        "torso_vel": torso_ang_vel,
        "humerus_vel": humerus_ang_vel,
        "hand_vel": hand_ang_vel
    }

def compute_static_offsets(static_file):
    _, markers, frame_rate, _, _ = load_c3d_file(static_file)
    kin = compute_torso_humerus_hand_kinematics(markers, frame_rate)

    if kin is None:
        print(f"WARNING: Could not compute static offsets from {static_file}")
        return {}

    mean_abd = np.mean(kin["shoulder_abduction"])
    mean_er = np.mean(kin["shoulder_external_rot"])

    print(f"Static Offsets -> Abduction: {mean_abd:.2f}, External Rotation: {mean_er:.2f}")

    return {"abd": mean_abd, "er": mean_er}



# ===================== Compute Static Offsets if available ===================== #
static_offsets = {}
if static_files:
    # Use the first static file found
    static_offsets = compute_static_offsets(static_files[0])
    print("Static Offsets:", static_offsets)

# ===================== MAIN PROCESSING LOOP ===================== #
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

for file_path in c3d_files:
    base_name = os.path.basename(file_path)
    # skip static file in main loop
    if "static" in base_name.lower():
        continue

    print(f"Processing {base_name}")
    try:
        c3d_obj, markers, frame_rate, n_frames, events = load_c3d_file(file_path)
    except Exception as e:
        print(f"Error loading {base_name}: {e}")
        continue

    # Heuristics or events to find foot contact & ball release
    fc_frame, br_frame = detect_foot_contact_and_ball_release(events, markers, frame_rate)

    # compute angles & velocities
    kin = compute_torso_humerus_hand_kinematics(markers, frame_rate, static_offsets)
    if kin is None:
        print(f"Skipping {base_name}; could not compute angles.")
        continue

    # Extract metrics from FC..BR
    fc_frame = max(fc_frame, 0)
    br_frame = min(br_frame, n_frames-1)
    if br_frame <= fc_frame:
        print(f"Skipping {base_name}: ball release <= foot contact.")
        continue

    # Peak velocities in the window
    torso_in_window = kin["torso_vel"][fc_frame:br_frame+1]
    hum_in_window   = kin["humerus_vel"][fc_frame:br_frame+1]
    hand_in_window  = kin["hand_vel"][fc_frame:br_frame+1]

    peak_torso = np.max(torso_in_window)
    peak_hum   = np.max(hum_in_window)
    peak_hand  = np.max(hand_in_window)

    # Max abduction overall
    abd_in_window = kin["shoulder_abduction"][fc_frame:br_frame+1]
    max_abd = np.max(abd_in_window)
    abd_fc = kin["shoulder_abduction"][fc_frame]

    # Shoulder external rotation at foot contact & max
    er_fc = kin["shoulder_external_rot"][fc_frame]
    er_in_window = kin["shoulder_external_rot"][fc_frame:br_frame+1]
    max_er = np.max(er_in_window)

    # Insert into DB
    # We can parse participant_name, pitch_date, trial_type from folder structure if desired
    # For example, folder structure: root\\Participant01\\2025-02-21\\Fastball1.c3d
    # We'll do a simple parse here:
    folder_parts = os.path.normpath(file_path).split(os.sep)
    # e.g. [ ..., 'Participant01_123', '2025-02-21_', 'Fastball1.c3d']
    participant_name = None
    pitch_date = None
    trial_type = "Unknown"

    if len(folder_parts) >= 3:
        participant_name = folder_parts[-3]  # e.g. 'Participant01_123'
        pitch_date = folder_parts[-2].replace("_", "")
        # trial_type might be derived from the filename
        trial_type = os.path.splitext(folder_parts[-1])[0]
    
    if not participant_name:
        participant_name = "Unknown"
    if not pitch_date:
        pitch_date = "Unknown"

    insert_sql = """
    INSERT INTO pitch_kinematics (
        file_name, participant_name, pitch_date, trial_type,
        foot_contact_frame, ball_release_frame,
        peak_torso_vel, peak_humerus_vel, peak_hand_vel,
        max_abduction, abd_foot_contact, er_foot_contact, max_er
    ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)
    """

    data_tuple = (
        base_name,
        participant_name,
        pitch_date,
        trial_type,
        fc_frame,
        br_frame,
        float(peak_torso),
        float(peak_hum),
        float(peak_hand),
        float(max_abd),
        float(abd_fc),
        float(er_fc),
        float(max_er)
    )
    cursor.execute(insert_sql, data_tuple)
    conn.commit()

conn.close()

print("Done processing all trials. Data stored in pitch_kinematics table.")


# ===================== BUILD DASH APP ===================== #
# We'll load external_stylesheets with your custom CSS
# Make sure you have the 'assets/custom.css' in your Dash project folder.
# Then you can do:
external_stylesheets = [dbc.themes.BOOTSTRAP, "assets/custom.css"]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.title = "Youth Pitch Design Kinematics"

# Layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Pitch Kinematics", className="text-center text-primary mb-4"),
                width=12)
    ]),
    dbc.Row([
        # We'll add drop-downs to select participant, date, pitch type, etc.
        # Then display a graph of velocities or angles from the DB.
        dbc.Col([
            html.Label("Participant"),
            dcc.Dropdown(id="participant-dropdown",
                         style={"margin-bottom": "20px"}),
            html.Label("Date"),
            dcc.Dropdown(id="date-dropdown",
                         style={"margin-bottom": "20px"}),
            html.Label("Trial Type"),
            dcc.Dropdown(id="trial-dropdown",
                         style={"margin-bottom": "20px"}),
            dbc.Button("Refresh Data", id="refresh-btn", color="primary", className="w-100")
        ], width=3),
        dbc.Col([
            dcc.Graph(id="velocity-graph")
        ], width=9)
    ])
], fluid=True)

@app.callback(
    Output("participant-dropdown", "options"),
    Output("participant-dropdown", "value"),
    Input("refresh-btn", "n_clicks")
)
def update_participants(_):
    # Load participants from DB
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_kinematics", conn)
    conn.close()
    participants = df["participant_name"].unique().tolist()
    options = [{"label": p, "value": p} for p in participants]
    val = participants[0] if participants else None
    return options, val

@app.callback(
    Output("date-dropdown", "options"),
    Output("date-dropdown", "value"),
    Input("participant-dropdown", "value")
)
def update_dates(participant):
    if not participant:
        return [], None
    conn = sqlite3.connect(DB_PATH)
    query = f"SELECT DISTINCT pitch_date FROM pitch_kinematics WHERE participant_name='{participant}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    dates = df["pitch_date"].unique().tolist()
    options = [{"label": d, "value": d} for d in dates]
    return options, (dates[0] if dates else None)

@app.callback(
    Output("trial-dropdown", "options"),
    Output("trial-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value")
)
def update_trial_types(participant, date):
    if not participant or not date:
        return [], None
    conn = sqlite3.connect(DB_PATH)
    query = f"""
    SELECT DISTINCT trial_type FROM pitch_kinematics
    WHERE participant_name='{participant}' AND pitch_date='{date}'
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    trials = df["trial_type"].unique().tolist()
    options = [{"label": t, "value": t} for t in trials]
    return options, (trials[0] if trials else None)

@app.callback(
    Output("velocity-graph", "figure"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value"),
    Input("trial-dropdown", "value")
)
def update_graph(participant, date, trial):
    if not participant or not date or not trial:
        return go.Figure()

    conn = sqlite3.connect(DB_PATH)
    query = f"""
    SELECT file_name, peak_torso_vel, peak_humerus_vel, peak_hand_vel
    FROM pitch_kinematics
    WHERE participant_name='{participant}' AND pitch_date='{date}' AND trial_type='{trial}'
    """
    df = pd.read_sql_query(query, conn)
    conn.close()

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=df["file_name"],
        y=df["peak_torso_vel"],
        name="Peak Torso Vel"
    ))
    fig.add_trace(go.Bar(
        x=df["file_name"],
        y=df["peak_humerus_vel"],
        name="Peak Humerus Vel"
    ))
    fig.add_trace(go.Bar(
        x=df["file_name"],
        y=df["peak_hand_vel"],
        name="Peak Hand Vel"
    ))

    fig.update_layout(barmode='group',
                      title="Peak Angular Velocities",
                      xaxis_title="Trial",
                      yaxis_title="Velocity (deg/s)")
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)


Found static files:
   A+ Static RH 2.c3d

Processing markers for joint angle calculations:
['L_SHOULDER', 'STERNUM', 'R_SHOULDER', 'C7', 'BICEP', 'TRICEP', 'RIGHT_LATERAL_ELBOW', 'RIGHT_MEDIAL_ELBOW', 'FOREARM', 'RIGHT_WRIST_RADIUS', 'RIGHT_WRIST_ULNA', 'RIGHT_HAND', 'WAIST']
Missing required markers! Skipping this trial.
Static Offsets: {}
Processing Crow Hop RH 4.c3d

Processing markers for joint angle calculations:
['L_SHOULDER', 'STERNUM', 'R_SHOULDER', 'C7', 'BICEP', 'TRICEP', 'RIGHT_LATERAL_ELBOW', 'RIGHT_MEDIAL_ELBOW', 'FOREARM', 'RIGHT_WRIST_RADIUS', 'RIGHT_WRIST_ULNA', 'RIGHT_HAND', 'WAIST']
Missing required markers! Skipping this trial.
Skipping Crow Hop RH 4.c3d; could not compute angles.
Processing Crow Hop RH 3.c3d

Processing markers for joint angle calculations:
['L_SHOULDER', 'STERNUM', 'R_SHOULDER', 'C7', 'BICEP', 'TRICEP', 'RIGHT_LATERAL_ELBOW', 'RIGHT_MEDIAL_ELBOW', 'FOREARM', 'RIGHT_WRIST_RADIUS', 'RIGHT_WRIST_ULNA', 'RIGHT_HAND', 'WAIST']
Missing required markers!

TypeError: The `dash_bootstrap_components.Button` component (version 1.7.1) with the ID "refresh-btn" received an unexpected keyword argument: `block`
Allowed arguments: active, children, className, class_name, color, disabled, download, external_link, href, id, key, loading_state, n_clicks, n_clicks_timestamp, name, outline, rel, size, style, target, title, type, value